In [1]:
import copy
import sys
import qgrid
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymatgen import Structure

sys.path.append('../../lib/')
from dftmanlib.pwscf import (pwinput_helper,
                             pwcalculation_helper,
                             pseudo_helper)
from dftmanlib.pwscf.workflow import EOSWorkflow
from dftmanlib.db import (init, load, store)
from dftmanlib.job import SubmitJob, submitjob_statuses, submit_status
from dftmanlib.matproj import mpquery_helper

PSEUDO_TABLE = '/data/tools/shared/dftman/pseudo_table.json'
# available pseudo families:
# ['SSSP_EFFICIENCY', 'SSSP_PRECISION', 'GBRV_US_LDA',
#  'GBRV_US_PBE', 'GBRV_US_PBEsol', 'DOJO_STANDARD_LDA_NC',
#  'DOJO_STANDARD_PBE_NC', 'DOJO_STANDARD_PBEsol_NC',
#  'DOJO_STRINGENT_LDA_NC', 'DOJO_STRINGENT_PBE_NC',
#  'DOJO_STRINGENT_PBAsol_NC']
PSEUDO_FAMILY = 'GBRV_US_PBE'

qgrid.enable()
MP_API_KEY = '0WqdPfXxloze6T9N'
db = init()

This database already exists! Loading instead.


## Materials Project Query

In [2]:
excluded_other = ['H', 'F']
lanthanides = ['La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu']
acinides = ['Ac', 'Th', 'Pa', 'U', 'Np', 'Pu',  'Am', 'Cm', 'Bk', 'Cf', 'Es', 'Fm', 'Md', 'No', 'Lr']

# cubic (Pm-3m), trigonal (R3m), tetragonal (P4mm), orthorhombic (Amm2) 
spacegroups = {221: 'cubic', 160: 'trigonal', 99: 'tetragonal', 38: 'orthorhombic'}

In [3]:
SPACEGROUP = 221

criteria = {
    'anonymous_formula': {'A': 1, 'B': 1, 'C': 3},
    'elements': {'$in': ['O'],
                 '$nin': lanthanides+acinides+excluded_other},
    'spacegroup.number': {'$in': list(spacegroups.keys())},
    'run_type': 'GGA',
    'e_above_hull': {'$lt': 0.01},
    'nsites': 5,
    'spacegroup.number': SPACEGROUP,
}

properties = []
m = mpquery_helper(criteria, properties, MP_API_KEY)
m.query()
m.display()

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defaultColumnWidth': 150, 'rowHeight': 28, 'enableColumnReorder': False, 'enableTextSelectionOnCells': True, 'editable': True, 'autoEdit': False, 'explicitInitialization': True, 'maxVisibleRows': 15, 'minVisibleRows': 8, 'sortable': True, 'filterable': True, 'highlightSelectedCell': False, 'highlightSelectedRow': True}, id='2b702f45-0fbe-4d61-a3f1-8202d266eec7', precision=5)

# K-point Convergence

### Base configuration

In [ ]:
STRUCTURE = m.df[md.df['material_id'] == 'mp-5229']['structure'].tolist()[0]
PSEUDO = pseudo_helper(STRUCTURE, PSEUDO_FAMILY,
                       PSEUDO_TABLE)
base_inputs = {
        'structure': STRUCTURE,

        'control': {
            'calculation': 'scf',
            'verbosity': 'high',
            'disk_io': 'none',
        },
        'system': {
            'ibrav': 0,
            'space_group': SPACEGROUP,
            'ecutwfc': None,
            'occupations': 'smearing',
            'degauss': 0.001,
            'smearing': 'mv',
        },
        'electrons': {
            'electron_maxstep': 500,
            'conv_thr': 1.0e-8,
        },
        'ions': {},
        'cell': {},
        'kpoints_mode': 'automatic',
        'kpoints_grid': (None, None, None),
        'kpoints_shift': (0, 0, 0),

        'pseudo': PSEUDO
    }

### Customize, store, and run individual calculations

In [ ]:
ecut = 35
kpoints = [10, 12, 14, 16, 18, 20, 22, 24]
kpoints_keys = []

for kpoint in kpoints:    
    inputs = copy.deepcopy(base_inputs)
    inputs['system']['ecutwfc'] = ecut
    inputs['kpoints_grid'] = (kpoint, kpoint, kpoint)
    
    runname = 'Alkpoint{}'.format(kpoint)
    calculation = pwcalculation_helper(**inputs, additional_inputs=list(PSEUDO.values()))
    job = SubmitJob(calculation, 'espresso-6.2.1_pw', runname=runname)
    
    key = db_store(job, db)
    kpoints_keys.append(key)
    
    job.submit()

print(kpoints_keys)

In [ ]:
kpoints_keys = ['c9b81d930928', '141600746c17', 'f70dafabd6e2', 'ce3cf7231023', 'a7ef1708c06b', '2991a881756c', 'a23c33c66f60', 'f774ec0fc3fb']

### Check status

In [ ]:
# individual jobs
job_statuses([db.SubmitJobs[key] for key in kpoints_keys])
# status as a whole
submit_status()

### Parse complete jobs and plot results

In [ ]:
complete_jobs = []
for key in kpoints_keys:
    job = db.SubmitJobs[key]
    if job.status == 'Complete':
        job.parse_output()
        complete_jobs.append(job)

data = {
    # energies per atom in eV
    'energies': [job.output.final_total_energy\
                 / len(job.input.structure.sites)\
                 for job in complete_jobs],
    'k_points': [job.input.kpoints_grid[0]\
                 for job in complete_jobs]
}
df = pd.DataFrame(data)
df.sort_values(by='k_points')
df['d_energies'] = [np.abs(df['energies'][i+1] - df['energies'][i])\
                    for i in range(len(df['energies'])-1)] + [np.nan]

fig = plt.figure()
fig.set_dpi(300)
ax = plt.gca()
plt.plot(df['k_points'], df['d_energies'], marker='o')
ax.axhline(0.005)
plt.xlabel('k-points')
plt.ylabel('Change in Energy per atom (eV)')
plt.show()
plt.close()

# Kinetic Energy Cutoff Convergence

### Base configuration

In [ ]:
STRUCTURE = m.result[0]['structure']
PSEUDO = pseudo_helper(STRUCTURE, PSEUDO_FAMILY,
                       PSEUDO_TABLE)
base_inputs = {
        'structure': STRUCTURE,

        'control': {
            'calculation': 'scf',
            'verbosity': 'high',
            'disk_io': 'none',
        },
        'system': {
            'ibrav': 0,
            'ecutwfc': None,
            'occupations': 'smearing',
            'degauss': 0.01,
            'smearing': 'mv',
        },
        'electrons': {
            'electron_maxstep': 500,
            'conv_thr': 1.0e-7,
        },
        'ions': {},
        'cell': {},
        'kpoints_mode': 'automatic',
        'kpoints_grid': (None, None, None),
        'kpoints_shift': (0, 0, 0),

        'pseudo': PSEUDO
    }

### Customize, store, and run individual calculations

In [ ]:
kpoint = 18
ecutwfcs = [20, 25, 30, 36, 40, 45, 50] # 35 is already in the database!
structure = m.result[0]['structure']
pseudo = pseudo_helper(structure, PSEUDO_FAMILY,
                       PSEUDO_TABLE)
ecutwfcs_keys = []
for ecutwfc in ecutwfcs:
    inputs = copy.deepcopy(base_inputs)
    inputs['system']['ecutwfc'] = ecutwfc
    inputs['kpoints_grid'] = (kpoint, kpoint, kpoint)
    
    runname = 'Alecut{}'.format(ecutwfc)
    calculation = pwcalculation_helper(**inputs, additional_inputs=list(pseudo.values()))
    job = SubmitJob(calculation, 'espresso-6.2.1_pw', runname=runname, ncpus=2)
    
    key = db_store(job, db)
    ecutwfcs_keys.append(key)
    
    job.submit()
    
print(ecutwfcs_keys)

In [ ]:
ecutwfcs_keys = ['df7215e5d23a', 'a40a6608a250', 'c3c89d562be0', '96a4f8026101', '69d8484cf8a5', 'ed1eec2087ca', '161e81152d34']

## Check status

In [ ]:
# individual jobs
job_statuses([db.SubmitJobs[key] for key in ecutwfcs_keys])
# status as a whole
submit_status()

## Parse complete jobs and plot results

In [ ]:

complete_jobs = []
for key in ecutwfcs_keys:
    job = db.SubmitJobs[key]
    if job.status == 'Complete':
        job.parse_output()
        complete_jobs.append(job)
        
data = {
    'energies': [job.output.final_total_energy\
                 for job in complete_jobs],
    'ecutwfcs': [job.input.sections['system']['ecutwfc']\
                 for job in complete_jobs]
}
df = pd.DataFrame(data)
df.sort_values(by='ecutwfcs')
df['d_energies'] = [np.abs(df['energies'][i+1] - df['energies'][i])\
                    for i in range(len(df['energies'])-1)] + [np.nan]

fig = plt.figure()
fig.set_dpi(300)
ax = plt.gca()
plt.plot(df['ecutwfcs'], df['d_energies'], marker='o')
ax.axhline(0.001)
plt.xlabel('ecutwfc (Ry)')
plt.ylabel('Change in Energy (eV)')
plt.show()
plt.close()